In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tqdm import tqdm
from keras.utils import to_categorical
from copy import copy
import tensorflow as tf

Using TensorFlow backend.


In [2]:
data_sample_class = pd.read_pickle('rdf2vec_data/data_classes_entities')

In [3]:
data_sample_vectors = pd.read_pickle('rdf2vec_data/data_vectors_entities')

In [4]:
data_sample_vectors[0]

array([ 0.06882069,  0.5259999 , -0.73323894, -0.43727952,  0.67447037,
        0.09834116, -0.46560106, -0.74799967,  0.02877666,  0.20510344,
        0.08196632, -0.02526115,  0.082765  , -0.41797048,  0.38921273,
        0.46458974,  0.47773632, -0.02372607,  0.0720956 , -0.24251032,
        0.06313052,  0.51026374,  0.6961665 ,  0.30973288,  0.31848267,
       -0.17720169, -0.2878234 ,  0.84214514,  0.6483063 ,  0.07255545,
       -0.29008368, -0.03906672,  0.60482574, -0.30301204, -0.39025444,
        0.06448072,  0.0690306 ,  0.04224071, -0.3442633 ,  0.07604404,
       -0.39567846, -0.09069009, -0.11476668,  0.32522783, -0.7187306 ,
       -0.58294624,  0.17802882, -0.4518249 , -0.03866209,  0.39672926,
       -0.5056423 ,  0.2016288 , -0.1805237 , -0.05329222,  0.78987116,
        0.56669897,  0.20621787,  0.1247419 , -0.08690415, -0.59782195,
        0.16341485, -0.29417554,  0.32382002,  0.22989944,  0.28220245,
       -0.38468555, -0.29087123,  0.17276445,  0.1912825 ,  0.00

In [5]:
data_sample_class[0]

['Cyclist', 'Person', 'Athlete', 'Agent']

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

y = mlb.fit_transform(data_sample_class)

In [7]:
y[0].shape

(455,)

In [8]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(np.array(data_sample_vectors),y,test_size=0.10, random_state=42)

x_train,x_val,y_train,y_val = train_test_split(x_train,y_train, test_size=0.2,random_state=42)

In [9]:
from keras.layers import Input, Dense,Dropout
from keras.layers.merge import concatenate
from keras.regularizers import l2
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint

In [12]:
input_data = Input(shape=(200,), name='input_data')

fc1 = Dense(256,activation='relu')(input_data)

fc2 = Dense(128, activation='relu')(fc1)

fc3 = Dense(64, activation='relu')(fc2)

fc3 = Dropout(.15)(fc3)

fc4 = Dense(455, activation='sigmoid')(fc3)

model_mlp = Model(input_data, fc4)

model_mlp.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model_mlp.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_data (InputLayer)      (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               51456     
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 455)               29575     
Total params: 122,183
Trainable params: 122,183
Non-trainable params: 0
_________________________________________________________________


In [13]:
checkpoint = ModelCheckpoint('weights-best-model-rdf2vec.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model_mlp.fit(x_train, y_train, batch_size=64, epochs=50,validation_data=(x_val,y_val),verbose=1,callbacks=callbacks_list)


Train on 253850 samples, validate on 63463 samples
Epoch 1/50
253850/253850 [==============================] - 24s 95us/step - loss: 0.0166 - acc: 0.9948 - val_loss: 0.0064 - val_acc: 0.9978
Epoch 2/50
253850/253850 [==============================] - 23s 92us/step - loss: 0.0062 - acc: 0.9979 - val_loss: 0.0048 - val_acc: 0.9984
Epoch 3/50
253850/253850 [==============================] - 23s 92us/step - loss: 0.0048 - acc: 0.9983 - val_loss: 0.0040 - val_acc: 0.9986
Epoch 4/50
253850/253850 [==============================] - 23s 91us/step - loss: 0.0042 - acc: 0.9986 - val_loss: 0.0038 - val_acc: 0.9987
Epoch 5/50
253850/253850 [==============================] - 23s 91us/step - loss: 0.0037 - acc: 0.9987 - val_loss: 0.0035 - val_acc: 0.9988
Epoch 6/50
253850/253850 [==============================] - 23s 91us/step - loss: 0.0034 - acc: 0.9988 - val_loss: 0.0034 - val_acc: 0.9989
Epoch 7/50
253850/253850 [==============================] - 23s 92us/step - loss: 0.0032 - acc: 0.9989 - val_

KeyboardInterrupt: 

In [71]:
model_mlp.load_weights('weights-best-model-rdf2vec.hdf5')
model_mlp.evaluate(x_test, y_test)

35258/35258 [==============================] - 2s 61us/step


[0.003353652922126277, 0.9990029591711781]

In [177]:
vector_test = data_sample_vectors[283238]

In [190]:
predict = model_mlp.predict(x_test)

In [200]:
predict = model_mlp.predict(vector_test.reshape(1,-1))

In [201]:
predict

array([[9.41784268e-32, 0.00000000e+00, 2.07388942e-33, 7.76002935e-27,
        1.08606229e-02, 9.31103374e-32, 2.26360542e-04, 9.99997973e-01,
        1.51999297e-36, 0.00000000e+00, 0.00000000e+00, 1.22188338e-23,
        3.86665419e-24, 4.32535952e-14, 2.91200045e-37, 4.35440638e-21,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.98189998e-24,
        1.02881044e-32, 3.48316305e-21, 4.59882735e-25, 0.00000000e+00,
        1.65570499e-37, 1.53912634e-08, 3.52500957e-24, 5.74304970e-36,
        0.00000000e+00, 9.02324796e-01, 1.51966771e-22, 1.06485653e-14,
        0.00000000e+00, 1.22198055e-24, 2.64439336e-07, 3.88348125e-29,
        8.80662827e-34, 2.33820671e-15, 0.00000000e+00, 9.85531479e-35,
        0.00000000e+00, 1.30209912e-10, 8.46138281e-24, 0.00000000e+00,
        3.97355499e-12, 1.26000280e-29, 1.53669976e-24, 1.76600735e-31,
        9.84302924e-20, 6.70482864e-38, 2.35148507e-34, 0.00000000e+00,
        4.61909351e-20, 7.60438355e-37, 7.08257498e-38, 3.561170

In [81]:
predict_list=list()
for items in predict:
    aux = list()
    for el in items:
        if el > .5:
            aux.append(1)
        else:
            aux.append(0)
    predict_list.append(aux)

In [38]:
labels = [str(i) for i in range(455)]

In [104]:
from sklearn.metrics import f1_score, average_precision_score, precision_score, recall_score


In [105]:
print(f1_score(y_test, np.array(predict_list), average='weighted'))
print(average_precision_score(y_test, predict_list, average='samples'))
print(precision_score(y_test, np.array(predict_list), average='weighted'))
print(recall_score(y_test, np.array(predict_list), average='weighted'))

/home/mobility/anaconda/envs/michael_env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mobility/anaconda/envs/michael_env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.9347092742811166
0.9002030878988591


/home/mobility/anaconda/envs/michael_env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.9497639579605351
0.9239992411307152


/home/mobility/anaconda/envs/michael_env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [102]:
type(predict_list), type(y_test)

(list, numpy.ndarray)

In [93]:
y_test[1000] == np.array(predict_list[1000])

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [89]:
y_test[100]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,